In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Sun Jun 13 06:02:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    36W / 250W |   4701MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
#import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,\
 Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [18]:
# (3) Create a sequential model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(32, kernel_size=(5, 5), input_shape=(100, 100, 3), activation='relu'))
model.add(Activation('relu'))
# Pooling 
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

# (4) Compile 
model.compile(loss='binary_crossentropy', optimizer='adam',\
 metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 96, 96, 32)        2432      
_________________________________________________________________
activation_27 (Activation)   (None, 96, 96, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_24 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 38, 38, 256)       991488    
_________________________________________________________________
activation_28 (Activation)   (None, 38, 38, 256)       0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 19, 19, 256)      

In [19]:
# Part 2 - Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/CNN/dataset/training_set',
                                                 target_size = (100, 100),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/CNN/dataset/test_set',
                                            target_size = (100, 100),
                                            batch_size = 32,
                                            class_mode = 'binary')


Found 8048 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
#callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

In [20]:
classifier = model.fit(training_set,
                         steps_per_epoch = len(training_set),
                         epochs = 25,
                         validation_data = test_set,    
                         validation_steps = len(test_set))

Epoch 1/25
252/252 [==============================] - 65s 244ms/step - loss: 1.0604 - accuracy: 0.5407 - val_loss: 0.7307 - val_accuracy: 0.5705
Epoch 2/25
252/252 [==============================] - 60s 236ms/step - loss: 0.6962 - accuracy: 0.6034 - val_loss: 0.7912 - val_accuracy: 0.5370
Epoch 3/25
252/252 [==============================] - 60s 236ms/step - loss: 0.6799 - accuracy: 0.6081 - val_loss: 0.7186 - val_accuracy: 0.6445
Epoch 4/25
252/252 [==============================] - 60s 238ms/step - loss: 0.6285 - accuracy: 0.6601 - val_loss: 0.7302 - val_accuracy: 0.5720
Epoch 5/25
252/252 [==============================] - 59s 236ms/step - loss: 0.5979 - accuracy: 0.6913 - val_loss: 0.6837 - val_accuracy: 0.6405
Epoch 6/25
252/252 [==============================] - 60s 239ms/step - loss: 0.5686 - accuracy: 0.7093 - val_loss: 0.6171 - val_accuracy: 0.6860
Epoch 7/25
252/252 [==============================] - 60s 238ms/step - loss: 0.5238 - accuracy: 0.7436 - val_loss: 0.6135 - val_ac

In [21]:
model.save("Alexnet.h5")
print("Saved model to disk")

Saved model to disk


In [22]:
import numpy as np
from keras.models import load_model
from keras.preprocessing import image

In [23]:
# load model
model_load = load_model('Alexnet.h5')

In [32]:
test_image = image.load_img('/content/drive/MyDrive/CNN/dataset/single_prediction/cat_or_dog_5.jpg', target_size = (100, 100))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model_load.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
print(prediction)

cat
